In [6]:
import torch
import numpy as np
import math
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
from sklearn.model_selection import train_test_split
import torch.nn as nn
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.metrics import r2_score
import random
import matplotlib as mpl
import os
import gc
import pandas as pd
import csv
from numpy import *
from torch.utils.tensorboard import SummaryWriter
from datetime import date
# Special imports

In [7]:
# We have downloaded this Excel file and reformatted it into a more parseable form
f = open("./AMP_dataset.txt", "r")
store_seq = []
seq_length = []
output_y = []
for x in f:
    line = list(x)
    store_seq.append(line[0:len(line)-3])
    output_y.append(int(line[-2]))
    seq_length.append(int(len(line)-3))
    # print(line[0:len(line)-3])

sequence = np.zeros((len(store_seq), max(seq_length)), dtype=object)

for i in range(len(store_seq)):
    sequence[i,0:seq_length[i]] = store_seq[i]

print('Max', max(seq_length), 'Min', min(seq_length))

Max 306 Min 27


**Converting to OHE**

In [8]:
# # different amino acids
amino_acid = ['A', 'T','G','C', 'X'] # X is the undetermined amino acid, so total length is 21
print('Number of unique amino acids are', np.shape(np.unique(amino_acid))[0])

np.save('../model/categorical_variables', amino_acid)

def onehotseq(sequence):
  seq_len = len(sequence)
  seq_en = np.zeros(( seq_len, np.shape(amino_acid)[0]))
  for i in range(seq_len):
    if (sequence[i] in amino_acid):
      pos = amino_acid.index(sequence[i])
      seq_en[i,pos] = 1     
    # else:
    #   pos = amino_acid.index('X')
    #   seq_en[i,pos] = 1
  return seq_en

### input sequence data
def one_hot_encoding(input_sequence, seq_length):
    ohe = np.zeros((input_sequence.shape[0], max(seq_length), len(amino_acid)))
    for i in range(input_sequence.shape[0]):
        seq_len = seq_length[i]
        seq_en = onehotseq(input_sequence[i,0:seq_len])
        ohe[i,0:seq_len,:] = seq_en
    return ohe

ohe = one_hot_encoding(sequence, seq_length)
seq_length = np.array(seq_length).reshape((len(seq_length),))
output_y = np.array(output_y).reshape((len(output_y),))


# idx = (seq_length>120)
# ohe = ohe[idx]
# seq_length = seq_length[idx]
# output_y = output_y[idx]

Number of unique amino acids are 5


### Below we are randomly choosing 1000 training, 100 test and valid examples. This is for showing interpretability quickly. 
#### feel free to keep all the data for interpretability


In [9]:
## Split the train and test
print('Total examples', ohe.shape)
x = np.arange(ohe.shape[0])
y = x 
X, x_test, Y, y_test = train_test_split( x, y, test_size=0.1923, random_state=10) ##[10,20,30]
x_train, x_valid, y_train, y_valid = train_test_split( X, Y, test_size=0.23809, random_state=10)

x_mut = x_test.copy()
_, x_test, _, _ = train_test_split( x_test, x_test, test_size=0.1, random_state=30) ##[10,20,30]
_, x_train, _, _ = train_test_split( x_train, x_train, test_size=0.3125, random_state=10) ##[10,20,30]
_, x_valid, _, _ = train_test_split( x_valid, x_valid, test_size=0.1, random_state=10) ##[10,20,30]

print(len(x_test), len(x_train), len(x_valid))
print('Mut', len(x_mut))

Total examples (5200, 306, 5)
100 1000 100
Mut 1000


In [10]:
np.save('./x_train', ohe[x_train])
np.save('./len_train', seq_length[x_train])
np.save('./y_train', output_y[x_train])

np.save('./x_valid', ohe[x_valid])
np.save('./len_valid', seq_length[x_valid])
np.save('./y_valid', output_y[x_valid])

np.save('./x_test', ohe[x_test])
np.save('./len_test', seq_length[x_test])
np.save('./y_test', output_y[x_test])
      
print(ohe[x_train].shape, seq_length[x_train].shape, output_y[x_train].shape)
print(ohe[x_valid].shape, seq_length[x_valid].shape, output_y[x_valid].shape) 
print(ohe[x_test].shape, seq_length[x_test].shape, output_y[x_test].shape) 

print(np.sum(output_y[x_test]))
# np.save('./store_data/x_train', ohe[x_train])
# np.save('./store_data/len_train', seq_length[x_train])
# np.save('./store_data/y_train', output_y[x_train])

# np.save('./store_data/x_valid', ohe[x_valid])
# np.save('./store_data/len_valid', seq_length[x_valid])
# np.save('./store_data/y_valid', output_y[x_valid])

# np.save('./store_data/x_test', ohe[x_test])
# np.save('./store_data/len_test', seq_length[x_test])
# np.save('./store_data/y_test', output_y[x_test])

(1000, 306, 5) (1000,) (1000,)
(100, 306, 5) (100,) (100,)
(100, 306, 5) (100,) (100,)
54
